In [29]:
import json
from web3 import Web3

tca = lambda x: w3.toChecksumAddress(x)

class StateChannel:
    
    def __init__(self, json_file="../project/build/contracts/StateChannel.json"):

        self.w3 = Web3(Web3.HTTPProvider("http://127.0.0.1:7545"))
        assert w3.eth.blockNumber
    
        # Solidity source code
        with open("../project/build/contracts/StateChannel.json", "r") as f:
            contract_data = json.load(f)
            
        address = tca(contract_data["networks"]["5777"]["address"])
        
        self.contract = self.w3.eth.contract(address=address, abi=contract_data["abi"])
        self.owner = tca(self.contract.functions.owner().call())
    
    def accepting_new_channels(self) -> bool:
        return self.contract.functions.accepting_new_channels().call()
        
    def set_accepting_new_channels(self, val: bool):
        transact_params = {"from": owner}
        fs = self.contract.transact(transact_params).changeAcceptanceStatus(val)
        self.w3.eth.waitForTransactionReceipt(fs)
        
    

In [44]:
st = StateChannel()
owner = st.owner

In [46]:
one = "0x0000000000000000000000000000000000000000000000000000000000000001"

In [45]:
st.set_accepting_new_channels(True)
print(st.accepting_new_channels())
st.set_accepting_new_channels(False)
print(st.accepting_new_channels())

True
False


In [47]:
msg = w3.sha3(text=one + owner + one)
signature = w3.eth.sign(owner, msg)

signature = signature.hex()

r = signature[:66]
s = '0x' + signature[66:130]
v = '0x' + signature[130:132]
v = int(v, 16)
v = v if v in [27, 28] else v + 27

r,s,v

('0x0e22082048b05d088ff506e206c8b162e569ac10953728d7aac9935a8a43d6ba',
 '0x1be21f39fd2d0782492fa809ef80f85a88de7700076103d13b0ff9264b7d4245',
 27)

In [48]:
cnt = st.contract

In [49]:
cnt.functions.verify(msg.hex(), v, r, s).call() == cnt.functions.owner().call()

True

In [52]:
cnt.functions.verify_receipt(1, owner, 1, v, r, s).call()

False